# LightSB I2I Implementation on BDD100K using the Shipped ALAE Encoder

#### Make sure the kernel is YOLOPX_LIGHTSB!

This notebook attempts a passthrough of the 

# Config
Setup YOLOPX path and namespace, setup LightSB path. Prepare everything we need to procure the data.

In [6]:
# Appends the YOLOPX directory to the path, so we can import the BDD100K dataloader
import os, sys

path = os.getcwd()
BASE_DIR = os.path.dirname(os.path.dirname(path))
print(BASE_DIR)
assert(os.path.basename(BASE_DIR) == "LightSB_YOLO")
YOLOPX_DIR = os.path.join(BASE_DIR, "", "models\YOLOPX")

sys.path.append(YOLOPX_DIR)
print(YOLOPX_DIR)

c:\Users\joche\Development\Github\LightSB_YOLO
c:\Users\joche\Development\Github\LightSB_YOLO\models\YOLOPX


In [7]:
from lib.utils import DataLoaderX

c:\Users\joche\anaconda3\envs\yolopx_lightsb\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
LIGHTSB_DIR = os.path.join(BASE_DIR, "", "LightSB")
sys.path.append(LIGHTSB_DIR)
print(LIGHTSB_DIR)

c:\Users\joche\Development\Github\LightSB_YOLO\LightSB


# Setup Config

In [9]:
from argparse import Namespace
from lib.config import cfg, update_config

args = Namespace()
args.iou_thres = 0.6
args.conf_thres = 0.001
args.weights = "../weights/epoch-195.pth"
args.modelDir = ''
args.logDir = 'runs/'

# annotations directory setup
args.da_seg_annotations = os.path.join(BASE_DIR, '', 'data/da_seg_annotations')
args.det_annotations = os.path.join(BASE_DIR, '', 'data/det_annotations')
args.images = os.path.join(BASE_DIR, '', 'data/images')
args.ll_seg_annotations = os.path.join(BASE_DIR, '', 'data/ll_seg_annotations')
update_config(cfg, args)

# Data
1. Get full training set from YOLOPX. 
2. Save training dataset to avoid reruns.
3. Use select_data to get separate domains, daytime and nighttime.

In [10]:
import lib.dataset as dataset
import pickle as pkl

pkl_dir = os.path.join(os.getcwd(), 'pkl_files', 'full_train_dataset.pkl')
skippable = os.path.exists(pkl_dir)
print(pkl_dir)
print("Skippable:", skippable)

full_train_dataset = dataset.BddDataset(
    cfg=cfg,
    is_train=True,
    inputsize=cfg.MODEL.IMAGE_SIZE,
    transform=None, 
    skip=skippable
)

if not skippable:
    with open(pkl_dir, 'wb') as f:
        pkl.dump(full_train_dataset.db, f)

else:
    with open(pkl_dir, 'rb') as f:
        full_train_dataset.db = pkl.load(f)

c:\Users\joche\anaconda3\envs\yolopx_lightsb\lib\site-packages\albumentations\__init__.py:13: UserWarning: A new version of Albumentations is available: 2.0.8 (you have 1.4.18). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
c:\Users\joche\anaconda3\envs\yolopx_lightsb\lib\site-packages\albumentations\core\composition.py:192: UserWarning: Got processor for bboxes, but no transform to process it.
  self._set_keys()


c:\Users\joche\Development\Github\LightSB_YOLO\notebooks\LightSB\pkl_files\full_train_dataset.pkl
Skippable: True


In [11]:
import copy
print('Full training set samples:', len(full_train_dataset.db))

train_day = copy.copy(full_train_dataset)
train_day.db = train_day.select_data(timeofday='daytime')
print('Daytime samples:', len(train_day.db))

train_night = copy.copy(full_train_dataset)
train_night.db = train_night.select_data(timeofday='night')
print('Night samples:', len(train_night.db))

print('Not included samples:', len(full_train_dataset.db) - len(train_day.db) - len(train_night.db))

Full training set samples: 70000
Daytime samples: 36800
Night samples: 28028
Not included samples: 5172


# Transform and Sample Images

In [12]:
# What a db entry looks like
print(train_day.db[0])

{'image': 'c:\\Users\\joche\\Development\\Github\\LightSB_YOLO\\data\\images\\train\\0000f77c-6257be58.jpg', 'label': array([[0.        , 0.15744014, 0.51558096, 0.24419134, 0.32413312],
       [0.        , 0.55320123, 0.4614185 , 0.3129219 , 0.30692723]]), 'mask': 'c:\\Users\\joche\\Development\\Github\\LightSB_YOLO\\data\\da_seg_annotations\\train\\0000f77c-6257be58.png', 'lane': 'c:\\Users\\joche\\Development\\Github\\LightSB_YOLO\\data\\ll_seg_annotations\\train\\0000f77c-6257be58.png', 'timeofday': 'daytime'}


In [13]:
from PIL import Image
# import torchvision.transforms as transforms
# resize = transforms.Compose([
#     transforms.Resize(cfg.MODEL.IMAGE_SIZE), 
#     transforms.ToTensor(),
#     transforms.ToPILImage()
#     ])

# img = resize(Image.open(train_day.db[28000]['image']).convert('RGB'))
img = Image.open(train_day.db[0]['image']).convert('RGB').resize((640,640))
img.show()

In [24]:
import cv2
from split_image import split_image

def resize_and_split(image_path, size=(32,32)):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (640, 640))
    height, width, _ = img.shape
    height = (int) (height/size[0])
    width = (int) (width/size[1])
    print(height, width)
    
    img = split_image(image_path, height, width, False, False, "../runs/")

resize_and_split(train_day.db[0]['image'])


20 20


# Model: LightSB

# Model: YOLOPX

# Evaluation